In [13]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar
import glob
import os

In [14]:
dataset = 'globcolour-cmems'
frequency = 'day'
variable = 'CHL1'
yearstart = 1999
yearend = 2023
monthorder = 'JULtoJUN'
averaging = 'wmean-latlon'

In [15]:
if dataset == 'globcolour-hermes':
    rootdir = '/projects/SOCCOM/datasets/GlobColour/'
    local = 'HERMES/merged/'+frequency+'/*/*/*/'
    mode = 'AV'
    filename = '*'+mode+'*'+variable+'*.nc'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        if 'CHL1_error' in ds.data_vars:
            ds = ds.drop(['CHL1_error'])
        ds = ds.sel({'lat':slice(-30,-65)}).expand_dims({'time':[time]}).rename({'CHL1_mean':variable})
        return ds
    
elif dataset == 'globcolour-cmems':
    rootdir = '/projects/SOCCOM/datasets/GlobColour/'
    local = 'CMEMS/OCEANCOLOUR_GLO_BGC_L4_MY_009_104-TDS/SouthernOcean_30Sto65S/'+frequency+'/bymonth.cmc/'
    filename = 'cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_*.nc'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        # This is a hack because of issues with data download
        ds = ds.where(ds['time.month']==ds['time.month'][0],drop=True).rename({'CHL':variable,'longitude':'lon','latitude':'lat'})
        return ds.copy()

print('Opening.')
paths = glob.glob(rootdir+local+filename)
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds = xr.open_mfdataset(paths,preprocess=preprocess,use_cftime=False)
ds['time'] = pd.to_datetime(ds['time'].values)
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,axes_dims_dict={'X':'lon','Y':'lat'},periodic='X')
ds['area'] = ds['dxC']*ds['dyC']

Opening.


In [16]:
ds

<xarray.Dataset>
Dimensions:          (time: 9617, lat: 840, lon: 8640, lon_left: 8640,
                      lat_left: 840)
Coordinates:
  * lat              (lat) float32 -64.98 -64.94 -64.9 ... -30.1 -30.06 -30.02
  * lon              (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time             (time) datetime64[ns] 1997-09-04 1997-09-05 ... 2024-01-02
  * lon_left         (lon_left) float32 -180.0 -180.0 -179.9 ... 179.9 180.0
  * lat_left         (lat_left) float32 -65.0 -64.96 -64.92 ... -30.08 -30.04
Data variables:
    CHL1             (time, lat, lon) float32 dask.array<chunksize=(27, 840, 8640), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(27, 840, 8640), meta=np.ndarray>
    flags            (time, lat, lon) float32 dask.array<chunksize=(27, 840, 8640), meta=np.ndarray>
    dxG              (lon_left, lat) float32 1.956e+03 1.959e+03 ... 4.004e+03
    dyG              (lon, lat_left) float32 nan 4.625e+03 ... 4.625e+03
    dxC              (lon, lat) float32 1.956e+03 1.959e+03 ... 4.004e+03
    dyC              (lon, lat) float32 4.624e+03 4.626e+03 ... 4.625e+03 nan
    rC               (lon, lat) float32 9.043e+06 9.06e+06 ... 1.851e+07 nan
    area             (lon, lat) float32 9.043e+06 9.06e+06 ... 1.851e+07 nan
Attributes: (12/92)
    Conventions:                       CF-1.8, ACDD-1.3
    DPM_reference:                     GC-UD-ACRI-PUG
    IODD_reference:                    GC-UD-ACRI-PUG
    acknowledgement:                   The Licensees will ensure that origina...
    citation:                          The Licensees will ensure that origina...
    cmems_product_id:                  OCEANCOLOUR_GLO_BGC_L4_MY_009_104
    ...                                ...
    time_coverage_resolution:          P1D
    time_coverage_start:               2023-11-20T15:24:55Z
    title:                             cmems_obs-oc_glo_bgc-plankton_my_l4-ga...
    westernmost_longitude:             -180.0
    westernmost_valid_longitude:       -180.0
    copernicus_marine_client_version:  0.10.3

In [17]:
ds = ds.chunk({'time':'auto'})
ds

<xarray.Dataset>
Dimensions:          (time: 9617, lat: 840, lon: 8640, lon_left: 8640,
                      lat_left: 840)
Coordinates:
  * lat              (lat) float32 -64.98 -64.94 -64.9 ... -30.1 -30.06 -30.02
  * lon              (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time             (time) datetime64[ns] 1997-09-04 1997-09-05 ... 2024-01-02
  * lon_left         (lon_left) float32 -180.0 -180.0 -179.9 ... 179.9 180.0
  * lat_left         (lat_left) float32 -65.0 -64.96 -64.92 ... -30.08 -30.04
Data variables:
    CHL1             (time, lat, lon) float32 dask.array<chunksize=(4, 840, 8640), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(4, 840, 8640), meta=np.ndarray>
    flags            (time, lat, lon) float32 dask.array<chunksize=(4, 840, 8640), meta=np.ndarray>
    dxG              (lon_left, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dyG              (lon, lat_left) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dxC              (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dyC              (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    rC               (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    area             (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
Attributes: (12/92)
    Conventions:                       CF-1.8, ACDD-1.3
    DPM_reference:                     GC-UD-ACRI-PUG
    IODD_reference:                    GC-UD-ACRI-PUG
    acknowledgement:                   The Licensees will ensure that origina...
    citation:                          The Licensees will ensure that origina...
    cmems_product_id:                  OCEANCOLOUR_GLO_BGC_L4_MY_009_104
    ...                                ...
    time_coverage_resolution:          P1D
    time_coverage_start:               2023-11-20T15:24:55Z
    title:                             cmems_obs-oc_glo_bgc-plankton_my_l4-ga...
    westernmost_longitude:             -180.0
    westernmost_valid_longitude:       -180.0
    copernicus_marine_client_version:  0.10.3

In [6]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds[variable] = ds[variable].fillna(0)*mask

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/regionmask/defined_regions/_natural_earth.py:471: FutureWarning: ``regionmask.defined_regions.natural_earth`` is deprecated. Please use ``regionmask.defined_regions.natural_earth_v4_1_0`` or ``regionmask.defined_regions.natural_earth_v5_0_0`` instead.
  warnings.warn(


In [7]:
# Extract spatial average for each year
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    if monthorder=='JULtoJUN':
        start = str(year-1)+'-07-01'
        end = str(year)+'-06-30'
        nleapday = 243 # for removing leap day (243 days after July 1)
    elif monthorder=='JANtoDEC':
        start = str(year-1)+'-01-01'
        end = str(year)+'-12-31'
        nleapday = 59 # for removing leap day (59 days after Jan 1)
    
    # Weighted spatial mean
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    
    # Interpolate to daily
    # (day frquency data has some missing days, so interpolate that too)
    alltimes = pd.date_range(start,end)
    xi = x.interp({'time':alltimes},kwargs={'fill_value':np.nan})
        
    # Remove leap-year day for simplicity
    if len(xi)==366:
        xi = xi[np.arange(len(xi))!=nleapday]
        
    # Put into dataset
    with ProgressBar():
        xi = xi.compute()
    ds_year.loc[{'year':year}]=xi.values

1999
[########################################] | 100% Completed | 24.91 s
2000
[########################################] | 100% Completed | 45.82 s
2001
[########################################] | 100% Completed | 53.95 s
2002
[########################################] | 100% Completed | 38.60 s
2003
[########################################] | 100% Completed | 27.92 s
2004
[########################################] | 100% Completed | 26.30 s
2005
[########################################] | 100% Completed | 57.40 s
2006
[########################################] | 100% Completed | 28.09 s
2007
[########################################] | 100% Completed | 41.86 s
2008
[########################################] | 100% Completed | 53.46 s
2009
[########################################] | 100% Completed | 49.97 s
2010
[########################################] | 100% Completed | 27.25 s
2011
[########################################] | 100% Completed | 46.22 s
2012
[###################

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)
outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,'nc'])

In [9]:
print('Saving to '+outpath)
if os.path.isfile(outpath):
    os.remove(outpath)
ds_year.to_netcdf(outpath)

Saving to ../../data/globcolour-cmems.day.wmean-latlon.JULtoJUN.1999-2023.nc


### By basin

In [10]:
atlind = 20
indwpac = 170
wpacepac = -130
epacatl = -70

cond_atlantic = (ds['lon']>=epacatl) & (ds['lon']<atlind)
cond_indian = (ds['lon']>=atlind) & (ds['lon']<indwpac)
cond_wpacific = (ds['lon']>=indwpac) | (ds['lon']<wpacepac)
cond_epacific = (ds['lon']>=wpacepac) & (ds['lon']<epacatl)

atlantic = ds[variable].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds[variable].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
wpacific = ds[variable].where(cond_wpacific,drop=True)
wpacific_area = ds['area'].where(cond_wpacific,drop=True)
epacific = ds[variable].where(cond_epacific,drop=True)
epacific_area = ds['area'].where(cond_epacific,drop=True)

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/

In [11]:
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Extract spatial average for each year
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
wpacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
epacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = pd.date_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # West Pacific
    print('wpacific')
    x = (wpacific.sel({'time':slice(start,end)})).weighted(wpacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        wpacific_year.loc[{'year':year}]=x.values
        
    # East Pacific
    print('epacific')
    x = (epacific.sel({'time':slice(start,end)})).weighted(epacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        epacific_year.loc[{'year':year}]=x.values

1999
atlantic
[########################################] | 100% Completed | 47.28 s
indian
[########################################] | 100% Completed | 26.73 s
wpacific
[########################################] | 100% Completed | 16.21 s
epacific
[########################################] | 100% Completed | 16.06 s
2000
atlantic
[########################################] | 100% Completed | 57.66 s
indian
[########################################] | 100% Completed | 44.51 s
wpacific
[########################################] | 100% Completed | 17.06 s
epacific
[########################################] | 100% Completed | 17.33 s
2001
atlantic
[########################################] | 100% Completed | 50.16 s
indian
[########################################] | 100% Completed | 55.17 s
wpacific
[########################################] | 100% Completed | 16.84 s
epacific
[########################################] | 100% Completed | 16.53 s
2002
atlantic
[############################

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)

for basin in ['atlantic','indian','wpacific','epacific']:
    outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,basin,'nc'])
    if basin=='atlantic':
        atlantic_year.to_netcdf(outpath)
    elif basin=='indian':
        indian_year.to_netcdf(outpath)
    elif basin=='wpacific':
        wpacific_year.to_netcdf(outpath)
    elif basin=='epacific':
        epacific_year.to_netcdf(outpath)